## Generation of light-curves for training NNs

The scripts is used to generate data sets containing light curves for training neural networks. The script contains:

1. Paths to directories and Input parameters
2. Extracting data from microlensing light-curves
3. Creating mock light-curves fpr a given magnification map and velocity
4. Storing the generated light curves 

Rewritten by: Soumya Shreeram <br>
Script adapted from: Eric Paic <br>
Date: 02nd March 2020

In [11]:
import numpy as np
import os,sys
import pickle as pkl
from astropy.io import fits
import glob

### 1. Paths to directories and Input parameters

In [15]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir)
print("Does directory exists? \n>",os.path.isdir(data_dir))

# setting the paths
datadir = data_dir + '/Data/'
scriptdir = data_dir + '/Data/scripts'
resultdir = data_dir + '/Data/results'
mapdir = data_dir + '/Data/maps/unconvolved/'
storagedir = data_dir + '/Data/maps/storage/'

Does directory exists? 
> True


Constants used to convert pixels to physical length

In [ ]:
einstein_r_1131= 2.5e16 #Einstein ring of RXJ1131 in cm assuming <M> = 0.3 M_0
einstein_r_03 = 3.414e16 #Einstein ring of QJ0158 in cm assuming <M> = 0.3 M_0
einstein_r_01 = einstein_r_03/np.sqrt(3) #Einstein ring of QJ0158 in cm assuming <M> = 0.1 M_0
einstein_r_001 = einstein_r_03/np.sqrt(30) #Einstein ring of QJ0158 in cm assuming <M> = 0.01 M_0

cm_per_pxl = 20*einstein_r_03/8192 #Find pixel scale assuming the map is 20R_E x 20R_E and 8192 pxl x 8192pxl
ld_per_pxl = cm_per_pxl/(30000000000*3600*24) #Light-day per pixel

### 2. Extracting the data microlensing light curve

In [ ]:
def good_draw_LC(params, map, time, cm_per_pxl , err_data, add_shut_noise=0):
    """
    Function to draw a light curve in a microlensing map
    @params:: is a list composed with the starting coordinaates of the trajectory, its velocity and direction.
    @map:: is the map used to draw the curve
    @time:: decides the sampling of the microlensing curve
    @cm_per_pxl:: gives the scale of the map which is calculated by knowing that a map is 20 R_e x 20 R_e
    @ err_data:: may be used to add experimental shutnoise to the resulting lightcurve if add_shut_noise=1

    @Returns the generated light curve and the coordinates of the starting and ending point of the trajectory, the latter is used only to display the trajectory.
    """    
    x_start = params[0]
    y_start = params[1]
    v = params[2]
    angle= params[3]


    #Projecting the velocity on x and y axis
    v_x = np.multiply(v, np.cos(angle))
    v_x = np.divide(np.multiply(100000 * 3600 * 24, v_x), cm_per_pxl)
    v_y = np.multiply(v, np.sin(angle))
    v_y = np.divide(np.multiply(100000 * 3600 * 24, v_y), cm_per_pxl)

    #Calculating the trajectory of the source in the map
    if v_x == 0:
        path_x = x_start * np.ones(len(time))
    else:
        path_x = np.add(np.multiply(np.add(time, -time[0]), v_x), x_start)
    if v_y == 0:
        path_y = y_start * np.ones(len(mjhd))
    else:
        path_y = np.add(np.multiply(np.add(time, -time[0]), v_y), y_start)

    path_x = path_x.astype(int)
    path_y = path_y.astype(int)

    # Checking if the trajectory doesn't go out of the map and gathering the value of the corresponding pixels which give the flux magnification (Hence 2.5*log() to convert in mag).
    if path_x[-1] <= len(map)-1 and path_y[-1] <= len(map)-1 and path_x[-1] >= 0 and path_y[-1] >= 0:
        if add_shut_noise:
            temp = np.add(np.multiply(-2.5, np.log10(map[path_y, path_x])),np.random.normal(0, np.mean(err_data), len(path_y)))
        else:
            temp = np.multiply(-2.5, np.log10(map[path_y, path_x]))
        lc = temp - temp[0] * np.ones(len(temp))
        return lc, [path_x[0], path_y[0], path_x[-1], path_y[-1]]


Reading from the file and getting the info

In [ ]:
f = open(datadir+"/microlensing/data/J0158_Euler_microlensing_upsampled_B-A.rdb","r")

f= f.read()
f=f.split("\n")
data = f[2:]

mjhd = np.array([]) # Time
mag_ml = np.array([]) # Magnitude of microlensing
err_mag_ml = np.array([]) # Error on the magnitude

for i,elem in enumerate(data):
    mjhd = np.append(mjhd,float(elem.split("\t")[0]))
    mag_ml = np.append(mag_ml, float(elem.split("\t")[1]))
    temp = elem.split("\t")[2]
    err_mag_ml= np.append(err_mag_ml,float(temp.split("\r")[0]))

### 3. Creating mock light-curves for a given magnification map and velocity

In [ ]:
list_r0 = [2,4,10,15,20,30,40,60,80,100] #radii of the source in pxl, the reference radius is 15pxl
n_curves = 100000 #number of generated curves
n_good_curves = 5000 #minimum number of curves that are not flat
for r0 in list_r0:
    print r0
    map = storagedir + "FML0.9/M0,3/mapA-B_fml09_R%s.fits" % (r0) #path to the magnification map
    img = fits.open(map)[0]
    final_map = img.data[:, :]
    v_source =500 #in km.s^-1
    v = v_source * np.ones(n_curves)

    # generating random starting coordinates and angles of the trajectories
    x = np.random.random_integers(200, len(final_map) - 200, n_curves)
    y = np.random.random_integers(200, len(final_map) - 200, n_curves)
    angle = np.random.uniform(0, 2 * np.pi, n_curves)

    params = []
    for i, elem in enumerate(x):
        params.append([x[i], y[i], v[i], angle[i]])

    lc = []
    i = 0
    j=0
    while i<n_good_curves:
        # Here mjhd is the time vector extracted form the data so if you use it to generate mock curves they will already have the season gaps.
        temp= good_draw_LC(params[j], final_map,mjhd, err_mag_ml,(20 * einstein_r_03) / 8192)
        j += 1
        if temp is not None:
            temp = temp[0]
            if np.amax(temp)-np.amin(temp) > 1: # we consider a curve is "not flat" if the difference between it min and max is over 1
                lc.append(temp)
                i+=1
                if i%1000 ==0:
                    print i

### 4. Storing light-curves in a pkl file

In [ ]:
with open(resultdir + 'LCmocks/simLC_A-B_n%s_v%s_R%s_M0,3.pkl' % (
     n_good_curves, v_source, r0), 'wb') as handle:
        pkl.dump((lc), handle, protocol=pkl.HIGHEST_PROTOCOL)